# PyISS v0.2 Feature Tour

This notebook mirrors the v0.1 test notebook but demonstrates the new composable operations introduced in v0.2.


In [1]:
# %pip install "pyiss @ git+https://github.com/ArnaudMath/pyiss.git"


In [2]:
import sys
from pathlib import Path

root = Path('..').resolve()
sys.path.insert(0, str(root / 'src'))

import pyiss as iss


In [3]:

# Seed with any valid ISS OPUS ID
set0 = iss.infer_set("co-iss-n1823513436")
# set0 = iss.infer_set("co-iss-w1540365932")
set0


In [4]:
set0.size, set0.available_filters, set0.time_window, set0.time_window.duration_s

(3,
 ['UV3', 'GRN', 'IR3'],
 TimeWindow(start=Timestamp('2015-10-14 10:25:26.342000+0000', tz='UTC'), end=Timestamp('2015-10-14 10:27:17.150000+0000', tz='UTC')),
 110.808)

In [5]:
f0 = set0.available_filters[0]
print("example filter:", f0)

# This should not work anymore. Check: IT DOESN"T WORK ANYMORE!
# print("opusid:", set0.opusid(f0))
# print("time1:", set0.time1(f0))
# print("target:", set0.target(f0))

# If the filter appears multiple times:
# print(set0.opusid(f0, all=True))


example filter: UV3


In [6]:
# Canonical table
set0.df


,opusid,time1,target,COISSfilter
0,co-iss-n1823513326,2015-10-14 10:25:26.342000+00:00,Enceladus,UV3
1,co-iss-n1823513381,2015-10-14 10:26:23.090000+00:00,Enceladus,GRN
2,co-iss-n1823513436,2015-10-14 10:27:17.150000+00:00,Enceladus,IR3


In [7]:
# Here you can check if it is indeed a bounded set. 
# It is a more elaborate version of df, but with more context:
# Row A shows the last images of the set before, 
# and row Z shows the first images of the very next set
set0.diagnostic_window()


,_role,opusid,time1,target,COISSfilter,dt_prev_s,dt_next_s
0,A,co-iss-n1823513218,2015-10-14 10:23:39.151000+00:00,Enceladus,IR3,NaN,107.191
1,set,co-iss-n1823513326,2015-10-14 10:25:26.342000+00:00,Enceladus,UV3,107.191,56.748
2,set,co-iss-n1823513381,2015-10-14 10:26:23.090000+00:00,Enceladus,GRN,56.748,54.060
3,set,co-iss-n1823513436,2015-10-14 10:27:17.150000+00:00,Enceladus,IR3,54.060,142.191
4,Z,co-iss-n1823513579,2015-10-14 10:29:39.341000+00:00,Enceladus,UV3,142.191,NaN


## v0.2: Composable operations


In [8]:
# Filter-slicing: returns a new ISSSet-like object
set_grn = set0.filter("GRN")
set_uv = set0.filter("UV3")
set_mix = set0.filter("GRN", "UV3")

set_grn.size, set_uv.size, set_mix.size


(1, 1, 2)

In [9]:
# Visual-only display with chained OPUS-like preview options
set0.show("GRN").image_size("medium").image_calibrated(True)


In [10]:
# Explicit non-visual downstream image type selection
set_cal = set_mix.image_type("CAL")
set_cal.selected_image_type


'CAL'

In [11]:
# Generic OPUS metadata access (no hardcoded field list)
print(set_mix.metadata("opusid", "time1"))


               opusid                            time1
0  co-iss-n1823513326 2015-10-14 10:25:26.342000+00:00
1  co-iss-n1823513381 2015-10-14 10:26:23.090000+00:00


In [12]:
# Preview images for a specific filter (defaults to full size, uncalibrated)
set0.show("UV3")
